# Connecting

In [1]:
import os

# setting working dir
os.chdir("../src")

In [2]:
from elasticsearch_db.elasticsearch import elastic_conection
from elasticsearch_db.elasticsearch import get_nlp_model

es = elastic_conection()

In [3]:
workspace_id = "dc1e7b3d-9137-4a20-a99c-d0d2029ef170"
workspace, exist = get_nlp_model(es, workspace_id=workspace_id)
exist

True

# Data Sourcing

In [10]:
from data_pipeline.data_sourcing import get_data

get_data?

Signature: get_data(workspace, es)
Docstring:
Get all the intents in the elasticsearch
workspace's index.

Parameters
----------
workspace: the workspace object
es: the elasticsearch conection instance

Returns
-------
list: a list of namedtuple Intents
    
Examples
--------

Raises
------

Notes
-----
File:      /usr/omni_nlu_api/src/data_pipeline/data_sourcing.py
Type:      function


In [11]:
data = get_data(workspace=workspace, es=es)
print(len(data))
print(data[0])

206
Intent(intent_name='Inativar_Posição', text_examples=['Como realizo a inativação de uma posição de minha estrutura?', 'Como realizo a reativação de uma posição em minha estrutura?', 'Em quanto tempo a inativação de uma posição é efetivada?', 'Gostaria de fazer a inativação de uma posição, como faço?', 'Realizei a inativação de uma posição e ela continua visível?'])


# Data Preprocessing

In [8]:
from data_pipeline.data_preprocessing import intents_with_few_data

intents_with_few_data?

Signature: intents_with_few_data(data)
Docstring:
Identify intents with less then 8 examples.
We expect at least 8 examples for each intent,
5 to train and 3 to test the ML models.

Parameters
----------
data: list of namedtuple representing Intents

Returns
-------
A list of indexes of the intents with less then 8 examples
    
Examples
--------

Raises
------

Notes
-----
File:      /usr/omni_nlu_api/src/data_pipeline/data_preprocessing.py
Type:      function


In [9]:
intents2create_artificial_examples = intents_with_few_data(data)
print(intents2create_artificial_examples)

[0, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 23, 24, 26, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 76, 77, 78, 79, 80, 83, 84, 85, 86, 87, 89, 91, 92, 95, 96, 97, 98, 100, 101, 103, 104, 106, 110, 111, 112, 113, 114, 115, 117, 118, 119, 121, 122, 124, 130, 131, 132, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 170, 171, 174, 176, 179, 180, 181, 183, 184, 185, 188, 190, 191, 192, 193, 197, 201, 202, 204, 205]


### Create artificial Examples

In [ ]:
import pt_core_news_lg

random.seed(42)

nlp = pt_core_news_lg.load()


def keep_token(token):
    return token.is_alpha and not (token.is_space or token.is_punct)


def create_new_sentence(sentence):
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc if keep_token(token)]).lower()

def create_artificial_examples(intent):
    quantity_of_examples = len(intent["examples"])
    messing_examples = quantity_of_examples - 8
    if messing_examples < 0 and messing_examples > -5:
        examples_ = random.sample(intent["examples"], abs(messing_examples))
        artificial_examples = [
                create_new_sentence(sentence["text"])
                for sentence in examples_
        ]
        intent["artificial_examples"] = artificial_examples
    elif messing_examples <= -5:
        examples_ = random.sample(intent["examples"], quantity_of_examples)
        artificial_examples = [
            {
                "constructed": create_new_sentence(sentence["text"]),
                "original": sentence["text"],
            }
            for sentence in examples_
        ]
        intent["artificial_examples"] = artificial_examples
    return intent
